In [20]:
import os
import getpass

In [21]:
MONGODB_ATLAS_CLUSTER_URI = getpass.getpass("MongoDB Atlas Cluster URI:")


In [23]:

# use openAI Embeddings so we need to set up our OpenAI API Key

# OPENAI_API_BASE="https://mzd-aoai-svc2.openai.azure.com/"
# OPENAI_API_VERSION="2023-05-15"
# OPENAI_API_TYPE="azure"


os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")


In [24]:
os.environ["OPENAI_API_BASE"] =  getpass.getpass("OpenAI API Base: ")


In [26]:
os.environ["OPENAI_API_VERSION"] =  getpass.getpass("OpenAI API Version: ")

In [27]:
os.environ["OPENAI_API_TYPE"] =  getpass.getpass("OpenAI API Type: ")

In [28]:

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain.document_loaders import TextLoader


In [29]:
loader = TextLoader("PVAmeeting.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()



In [32]:
from pymongo import MongoClient

# initialize MongoDB python client
client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)

db_name = "langchain_db"
collection_name = "langchain_col"
collection = client[db_name][collection_name]
index_name = "langchain_demo"

# insert the documents in MongoDB Atlas with their embedding
docsearch = MongoDBAtlasVectorSearch.from_documents(
    docs, embeddings, collection=collection, index_name=index_name
)

# perform a similarity search between the embedding of the query and the embeddings of the documents
query = "What should we do with registration related data"
docs = docsearch.similarity_search(query)

In [33]:
# print(docs[0].page_content)

In [34]:
# Instantiate the vector store directly

# initialize vector store
vectorstore = MongoDBAtlasVectorSearch(
    collection, OpenAIEmbeddings(), index_name=index_name
)

# perform a similarity search between a query and the ingested documents
query = "What did the president say about Ketanji Brown Jackson"
docs = vectorstore.similarity_search(query)

print(docs[0].page_content)

IndexError: list index out of range